In [226]:

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline



from datetime import datetime
import math
from sklearn.preprocessing import LabelEncoder
#machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve,accuracy_score,mean_squared_log_error
from sklearn.metrics import roc_curve, auc,roc_auc_score,mean_squared_error
import xgboost as xgb
from numpy.linalg import inv
from xgboost.sklearn import XGBClassifier,XGBRegressor
from sklearn.kernel_ridge import KernelRidge
import matplotlib.dates
import operator
from quilt.data.ResidentMario import missingno_data 
import missingno as msno
import tensorflow as tf


In [227]:
train = pd.read_csv("transformed_train1.csv")
test  = pd.read_csv("transformed_test1.csv")

In [228]:
itrain,itest = train_test_split(range(train.shape[0]),train_size=0.8,random_state = 2017)
mask = np.ones(train.shape[0],dtype='int')
mask[itrain] = 1
mask[itest] = 0
mask = (mask==1)

In [229]:
train.head()

,bandgap_energy_ev,formation_energy_ev_natom,id,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,number_of_total_atoms,...,clusters_of_lattices_9,area,area_bucket,atomic_density,nota_10.0,nota_20.0,nota_30.0,nota_40.0,nota_60.0,nota_80.0
0,3.4387,0.0680,1,90.0026,90.0023,90.0017,9.9523,8.5513,9.1775,80.0,...,1,73.732755,2,0.102426,0,0,0,0,0,1
1,2.9210,0.2490,2,90.0186,89.9980,120.0025,6.1840,6.1838,23.6287,80.0,...,0,127.626563,5,0.088537,0,0,0,0,0,1
2,2.7438,0.1821,3,90.9688,91.1228,30.5185,9.7510,5.6595,13.9630,40.0,...,0,42.049778,1,0.051910,0,0,0,1,0,0
3,3.3492,0.2172,4,89.9888,90.0119,120.0017,5.0036,5.0034,13.5318,30.0,...,0,59.629688,2,0.088556,0,0,1,0,0,0
4,1.3793,0.0505,5,89.9960,90.0006,119.9893,6.6614,6.6612,24.5813,80.0,...,0,143.093313,5,0.073344,0,0,0,0,0,1


In [230]:
features = ['lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang', 'number_of_total_atoms','spacegroup','one_to_two', 'one_to_three', 'two_to_three',
       'bucket_sg_ratio_1', 'bucket_sg_ratio_2', 'clusters_of_lattices',
       'clusters_of_lattices_0', 'clusters_of_lattices_1',
       'clusters_of_lattices_2', 'clusters_of_lattices_3',
       'clusters_of_lattices_4', 'clusters_of_lattices_5',
       'clusters_of_lattices_6', 'clusters_of_lattices_7',
       'clusters_of_lattices_8', 'clusters_of_lattices_9','area_bucket', 'atomic_density', 'nota_10.0', 'nota_20.0', 'nota_30.0',
       'nota_40.0', 'nota_60.0', 'nota_80.0']

In [231]:
target1 = 'bandgap_energy_ev'
target2 = 'formation_energy_ev_natom'

In [232]:
train[target1] = np.log1p(train[target1])
train[target2] = np.log1p(train[target2])

In [233]:
#trying to use tensor flow regression implementation for guessing outputs

In [254]:
#parameters
learning_rate = 0.1
training_epochs = 5000
display_step = 200

In [235]:
def normalize_data_frame(df):
    df_norm = (df - df.mean()) / (df.max() - df.min())
    return df_norm

In [236]:
train_X = train[mask]
validate_X = train[~mask]


train_Y1 = train_X[target1].as_matrix()
train_Y2 = train_X[target2].as_matrix()
train_X = train_X[features].as_matrix()


validate_Y1 = validate_X[target1].as_matrix()
validate_Y2 = validate_X[target2].as_matrix()
validate_X = validate_X[features].as_matrix()

test_X =  test[features].as_matrix()


In [237]:
train_X = normalize_data_frame(train_X)
validate_X = normalize_data_frame(validate_X)
test_X = normalize_data_frame(test_X)

In [238]:
n_samples = train_X.shape[0]
n_features = train_X.shape[1]

In [239]:
n_samples,n_features




(1918, 32)

In [240]:
train_X = train_X.T
validate_X = validate_X.T
test_X = test_X.T

In [241]:
train_Y1 = train_Y1.reshape(1,-1)
train_Y2 =  train_Y2.reshape(1,-1)
validate_Y1 = validate_Y1.reshape(1,-1)
validate_Y2 = validate_Y2.reshape(1,-1)

In [242]:
train_Y1_test.shape

(1, 1918)

In [255]:
def deep_net_model(X_data,num_features):
   
    
    W1 = tf.Variable(tf.random_normal([1024,num_features]))
    B1 = tf.Variable(tf.zeros([1024,1]))
    layer_1 = tf.add(tf.matmul(W1,X_data), B1)
    layer_1 = tf.nn.relu(layer_1)
    
    print(layer_1.shape)
    
  
    
        
    W2 = tf.Variable(tf.random_normal([512,1024]))
    B2 = tf.Variable(tf.zeros([512,1]))
    layer_2 = tf.add(tf.matmul(W2,layer_1), B2)
    layer_2 = tf.nn.relu(layer_2)
    
    W3 = tf.Variable(tf.random_normal([64,512]))
    B3 = tf.Variable(tf.zeros([64,1]))
    layer_3 = tf.add(tf.matmul(W3,layer_2), B3)
    layer_3 = tf.nn.relu(layer_3)
    print(layer_3.shape)

    W4 = tf.Variable(tf.random_normal([2,64]))
    B4 = tf.Variable(tf.zeros([2,1]))
    layer_4 = tf.add(tf.matmul(W4,layer_3), B4)
    layer_4 = tf.nn.relu(layer_4)
    
    WO = tf.Variable(tf.random_normal([1,2]))
    BO = tf.Variable(tf.zeros([1,1]))
    output = tf.add(tf.matmul(WO,layer_4), BO)
    
    print(output.shape)
    
    return output
    

In [259]:
xs = tf.placeholder("float")
ys = tf.placeholder("float")
output = deep_net_model(xs,32)
cost =  tf.reduce_mean(tf.square(tf.transpose(output)-tf.transpose(ys)))
train_optimizer = tf.train.AdamOptimizer(0.05).minimize(cost)
c_train = []
c_validation = []

(1024, ?)
(64, ?)
(1, ?)


In [260]:
train_X[:,6]

array([ 0.31879073,  0.31871232,  0.45087223, -0.05021896, -0.05021896,
        0.02705461,  0.2746621 ,  0.77686474, -0.030057  ,  0.03061246,
        0.03062073, -0.07776081, -0.07335552, -0.06895024, -0.07776081,
       -0.07776081, -0.07335552, -0.07776081, -0.07776081, -0.07776081,
       -0.07776081, -0.07776081, -0.07776081, -0.07776081, -0.05573438,
       -0.07738187, -0.07776081, -0.07776081, -0.07776081, -0.07776081,
       -0.07776081, -0.07335552])

In [261]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for i in range(1,2000):
        sess.run([cost,train_optimizer],feed_dict = {xs:train_X,ys:train_Y2})
        
        c_train.append(sess.run([cost],feed_dict = {xs:train_X,ys:train_Y2}))
        c_validation.append(sess.run([cost],feed_dict = {xs:validate_X,ys:validate_Y2}))
        if(i % display_step == 0):
            print("epoch  :", i, " cost: ", c_train[i-1])
            pred_validation = sess.run(output,feed_dict = {xs:validate_X})
            pred_train      = sess.run(output,feed_dict = {xs:train_X})
            print(np.sqrt(mean_squared_error(pred_train,train_Y2)))
            print(np.sqrt(mean_squared_error(pred_validation,validate_Y2)))
    
        

epoch  : 200  cost:  [0.20932114]
0.457516267083
0.457557250492
epoch  : 400  cost:  [0.17943984]
0.423603406185
0.423660581004
epoch  : 600  cost:  [0.14562197]
0.381604508819
0.381685793041
epoch  : 800  cost:  [0.11248385]
0.335386144271
0.335501076053
epoch  : 1000  cost:  [0.082991622]
0.288082656007
0.288243464588
epoch  : 1200  cost:  [0.058753677]
0.242391537972
0.24261414157
epoch  : 1400  cost:  [0.040244944]
0.200611452244
0.20091608132
epoch  : 1600  cost:  [0.027094616]
0.164604423694
0.165014739797
epoch  : 1800  cost:  [0.018413659]
0.135696936141
0.136235433306
